In [1]:
from LanguageModel import LanguageModel, NGramModel, NGramCounts, Corpus

In [2]:
LM = LanguageModel(corpus="Corpus/", verbose=True)

Reading Files:   0%|          | 0/5 [00:00<?, ?it/s]

Parsing XML:   0%|          | 0/5 [00:00<?, ?it/s]

Building Sentences:   0%|          | 0/5 [00:00<?, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Paragraph: 0it [00:00, ?it/s]

Counting x counts:   0%|          | 0/3408 [00:00<?, ?it/s]

Calculating Probabilities:   0%|          | 0/8285 [00:00<?, ?it/s]

In [3]:
LM.Perplexity(n=3, verbose=True)

Counting x counts:   0%|          | 0/3408 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/3408 [00:00<?, ?it/s]

Calculating Probabilities:   0%|          | 0/53759 [00:00<?, ?it/s]

Calculating Perplexity: 0it [00:00, ?it/s]

mpf('1.8419493042233952')

In [8]:
generated = LM.GenerateSentence(n=1, verbose=True)
for w in generated:
    print(w, end=' ')

print(".")

l nitkellem l kv kontra il qegħdin kredibilita li billi l jara tad hija jippreżenta l ta jirriforma pajjiż ma cassar meta kont george qiegħed .


In [9]:
generated = LM.GenerateSentence(verbose=True)
for w in generated:
    print(w, end=' ')

print(".")

9505 .


In [12]:
generated = LM.GenerateSentence(n=3, verbose=True)
for w in generated:
    print(w, end=' ')

print(".")

the deputy speaker onor helen d deċiżjoni li malta hija readable mill ogħla rati tas sisa fuq tabakk fosthom is surveys f tabelli 53 sa 57 .


In [15]:
generated = LM.GenerateSentence(n=4, verbose=True)
for w in generated:
    print(w, end=' ')

print(".")

interruzzjonijiet allura għalfejn noqogħdu ndaħħlu dawn jinbidlu fl ahjar interess tas servizz ġdid għal 25 ta lulju 2006 ġew konfermati .
